In [ ]:
import urllib2
from bs4 import BeautifulSoup
from urllib2 import HTTPError
import re
import pandas as pd
import csv
import codecs
import os


In [ ]:
# Receives htlm text
# Returns a list of all urls that are tagged a and start with 'http://'
def find_urls(data):
    BS = BeautifulSoup(data)
    urls = BS.findAll('a')
    url_list = []
    for link in urls:
        try:
            url_list.append(link['href'])
        except KeyError:
            pass
            #print link.attrs
    url_list = [url for url in url_list if re.match('http://.*', url, ) != None]
    return url_list

In [ ]:
def extract_article(data):
    text = ''
    BS = BeautifulSoup(data)
    try:
        BS=BS.article
        if BS == None:
            return text
        for child in BS.children:
            if child.get('itemprop')=='articleBody':
                s = child.findAll('p')
                for parag in s:
                    text = text + parag.get_text() 
    except AttributeError:
        return text  
    return text


In [ ]:
def clean_url(url):
    m = re.search("(https?://[a-zA-Z0-9.\/]+)", url)
    return m.group(1)

In [ ]:
def filter_urls(urls, current_url, already_visited = set([])):
    to_return = []
    for u in urls:
        u = clean_url(u)
        if u not in already_visited:
            domain_lst = u[7:].split('.')
            curr_domain = current_url[7:].split('.')
            if domain_lst[0] == curr_domain[0] and domain_lst[1] == curr_domain[1]:
                to_return.append(u)
    return to_return

        

In [ ]:
def arcticle_crawler(root_url, max_articles = 20):
    to_visit = []
    already_visited = set([])
    found_articles = 0
    to_visit.append(root_url)
    while to_visit and max_articles-found_articles>0:
        #print max_articles
        url = to_visit.pop(0)
        if url not in already_visited:
            try:
                site = urllib2.urlopen(url)
                text = ''
                if site.getcode() == 200:
                    data =''
                    for l in site:
                        data = data+l
                    urls = find_urls(data)
                    article = extract_article(data)
                    if(len(article)>0):
                        print "Found text of size: " + str(len(article))
                        found_articles = found_articles+1
                        print url
                        with codecs.open('article' + str(found_articles) + '.txt', "w", "utf-8-sig") as temp:
                            temp.write(article)
                    to_visit = to_visit + filter_urls(urls, url, already_visited)
            except HTTPError:
                pass
            already_visited.add(url)
            print "Still have " + str(len(to_visit)) +" urls to visit, found: "+str(found_articles)+" articles." 

    #return already_visited

In [ ]:
url = 'http://www.elmundo.es/espana/2016/03/15/56e7ffa922601d0c038b45ca.html'

arcticle_crawler('http://www.elmundo.es/espana', max_articles=5)